In [4]:
import requests
import os
import json
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
DATAESR_RNSR_INSTITUTIONS = os.getenv('DATAESR_RNSR_INSTITUTIONS')
DATAESR_MATCH_URL = os.getenv('DATAESR_MATCH_URL')
DATAESR_GET_URL = os.getenv('DATAESR_GET_URL')
DATAESR_AUTH = os.getenv('DATAESR_AUTH')

In [9]:
data = requests.get(DATAESR_RNSR_INSTITUTIONS, headers={"Authorization": DATAESR_AUTH}).json().get('data')

In [10]:
df = pd.DataFrame.from_dict(data)
df['RESULT'] = None
df

,rnsr_key,name,acronym,kind,rnsr_siret,rnsr_uai,end_date,start_date,RESULT
0,328,Institut Pasteur,INSTITUT PASTEUR,FONDATION,77568489700017,0755366A,None,2008-07-03T00:00:00,None
1,332,Laboratoire national de métrologie et d'essais,LNE,EPIC,31332024400012,0755503Z,None,1900-01-01T00:00:00,None
2,455,Agence nationale de sécurité sanitaire de l'al...,ANSES,EPA,13001202400183,0942357B,None,2010-07-01T00:00:00,None
3,301,Centre national de la recherche scientifique,CNRS,EPST,18008901303720,0753639Y,None,1900-01-01T00:00:00,None
4,305,Institut national de la recherche en agronomie,INRA,EPST,18007003901804,0755361V,2019-12-31T00:00:00,1900-01-01T00:00:00,None
...,...,...,...,...,...,...,...,...,...
655,728,BIOASTER,BIOASTER,FONDATION,75118352600035,None,None,2012-04-17T00:00:00,None
656,828,UNIVERSITÉ KAZAKHE NATIONALE AL-FARABI,KAZNU,UNIV_ETR,None,None,None,1934-01-01T00:00:00,None
657,829,Oncovita,OVTA,PRIVE,80947932200012,None,None,2015-01-29T00:00:00,None
658,832,ESIEE-IT - CCI PARIS ILE DE FRANCE - Pontoise,ESIEE-IT PONTOISE,AUT_ETAB,88931074400020,0952291Z,None,2020-09-01T00:00:00,None


In [15]:
headers = {"Authorization": DATAESR_AUTH}
for i, inst in enumerate(df['rnsr_siret']):
    try:
        r = requests.get(DATAESR_MATCH_URL + inst, headers=headers)
        if r.ok:
            if r.json().get('hits', 0) == 1:
                getr = requests.get(DATAESR_GET_URL + r.json().get('data', {})[0].get('id'), headers=headers)
                if getr.ok and getr.json().get('rnsr_key') == df['rnsr_key'][i]:
                    df['RESULT'][i] = "OK"
                else:
                    df['RESULT'][i] = "FOUND IN DATAESR BUT NO RNSR_KEY ATTRIBUTED"
            if r.json().get('hits', 0) == 0:
                df['RESULT'][i] = "NOT FOUND IN DATAESR"
            if r.json().get('hits', 0) > 1:
                df['RESULT'][i] = "MORE THAN ONE HIT"
        else:
            df['RESULT'][i] = "ERROR FETCHING DATAESR"
    except:
        df['RESULT'][i] = "NO SIRET PROVIDED"
df

,rnsr_key,name,acronym,kind,rnsr_siret,rnsr_uai,end_date,start_date,RESULT
0,328,Institut Pasteur,INSTITUT PASTEUR,FONDATION,77568489700017,0755366A,None,2008-07-03T00:00:00,OK
1,332,Laboratoire national de métrologie et d'essais,LNE,EPIC,31332024400012,0755503Z,None,1900-01-01T00:00:00,OK
2,455,Agence nationale de sécurité sanitaire de l'al...,ANSES,EPA,13001202400183,0942357B,None,2010-07-01T00:00:00,OK
3,301,Centre national de la recherche scientifique,CNRS,EPST,18008901303720,0753639Y,None,1900-01-01T00:00:00,OK
4,305,Institut national de la recherche en agronomie,INRA,EPST,18007003901804,0755361V,2019-12-31T00:00:00,1900-01-01T00:00:00,NOT FOUND IN DATAESR
...,...,...,...,...,...,...,...,...,...
655,728,BIOASTER,BIOASTER,FONDATION,75118352600035,None,None,2012-04-17T00:00:00,FOUND IN DATAESR BUT NO RNSR_KEY ATTRIBUTED
656,828,UNIVERSITÉ KAZAKHE NATIONALE AL-FARABI,KAZNU,UNIV_ETR,None,None,None,1934-01-01T00:00:00,NO SIRET PROVIDED
657,829,Oncovita,OVTA,PRIVE,80947932200012,None,None,2015-01-29T00:00:00,FOUND IN DATAESR BUT NO RNSR_KEY ATTRIBUTED
658,832,ESIEE-IT - CCI PARIS ILE DE FRANCE - Pontoise,ESIEE-IT PONTOISE,AUT_ETAB,88931074400020,0952291Z,None,2020-09-01T00:00:00,NOT FOUND IN DATAESR


In [17]:
df['RESULT'].value_counts()

OK                                             523
NO SIRET PROVIDED                               81
FOUND IN DATAESR BUT NO RNSR_KEY ATTRIBUTED     26
NOT FOUND IN DATAESR                            18
MORE THAN ONE HIT                               12
Name: RESULT, dtype: int64

In [18]:
df[df.RESULT=='FOUND IN DATAESR BUT NO RNSR_KEY ATTRIBUTED'][['rnsr_key', 'name']].drop_duplicates()

,rnsr_key,name
78,184,Institut d'études politiques de Paris - Fondat...
84,432,Institut français des sciences et technologies...
108,772,Université Toulouse Capitole EPE
142,769,Université de Rennes (EPE)
272,766,Institut Agronomique néo-Calédonien
366,839,Université de technologie de Tarbes
369,385,Centre régional de lutte contre le cancer - Ce...
473,782,Institut Catholique de Toulouse
474,825,Pôle Léonard de Vinci (ALDV)
479,598,GROUPE HOSPITALIER UNIVERSITAIRE PARIS psychia...


In [19]:
df[df.RESULT=='NOT FOUND IN DATAESR'][['rnsr_key', 'name']].drop_duplicates()

,rnsr_key,name
4,305,Institut national de la recherche en agronomie
13,186,AgroParisTech (Inst Sc et Ind du Vivant et Env...
59,833,Université Marie et Louis Pasteur
199,32,Ecole Nationale Supérieure des Mines d'Alès
209,834,Université Bourgogne Europe (EPE)
237,135,Ecole Nationale du Génie de l'Eau et de l'Envi...
288,96,Institut National Horticulture et Paysage Ange...
318,265,IUFM Guyane
321,262,Université Pascal Paoli Corse
455,770,Ecole d'économie et de sciences sociales quant...
